In [52]:
# Selected variables + LR Model with OHE & Polynomial
# Score: 1.5M

# 1. EDA and Cleaning

In [53]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score

In [54]:
# Read in Data
housing = pd.read_csv('datasets/train.csv')
housing_test = pd.read_csv('datasets/test.csv')

### Read through data dictionary, decide on which variables to use

In [55]:
# Remove spaces in col names b/c data dictionary col names do not include spaces
housing.columns = [n.replace(" ", "") for n in housing.columns]

# Save var names in txt file
# Create function to get variable names into list from txt file
# https://stackoverflow.com/questions/23372086/how-would-i-read-only-the-first-word-of-each-line-of-a-text-file
def get_var_name(txt_file):
    vars = []
    with open(txt_file, 'r') as f:
        for line in f:
            vars.append(line.split(None, 1)[0][:-1])
    return vars

init_vars = get_var_name('datasets/initial_vars.txt')
housing_init = housing[init_vars]

### Null values

In [56]:
# Check for NA values. 
# housing_init.isna().mean()
# Replace NAs with 0. They are not missing values, but 0 (i.e. basement bathroom is 0 not NA if there is no basement)
# https://www.geeksforgeeks.org/replace-nan-values-with-zeros-in-pandas-dataframe/
housing_init = housing_init.fillna(0)

### Variable Transformations

#### Add 'age' of home to be time since build or latest remodel. Assume data is from 2011

In [57]:
# Calculate age as 2011 - year of latest remodel or build
# housing_init['YearRemod/Add'].groupby(housing_init['YearRemod/Add']).count()
housing_init['Age'] = 2011 - housing_init['YearRemod/Add']
housing_init.drop(columns =['YearRemod/Add'], inplace= True)

In [58]:
# Ages range from 1-61 years, which makes sense 
housing_init['Age'].describe()

count    2051.000000
mean       26.809849
std        21.036250
min         1.000000
25%         7.000000
50%        18.000000
75%        46.500000
max        61.000000
Name: Age, dtype: float64

#### Add 'TotalBaths' as new variable, adding up basement full bathrooms, basement half bathrooms, full bathrooms above grade, and half bathrooms above grade. 

In [59]:
# Create new column in dataset for total number of bathrooms
housing_init['TotalBaths'] = housing_init['BsmtFullBath'] + 0.5*housing_init['BsmtHalfBath'] + housing_init['FullBath'] + 0.5*housing_init['HalfBath']
housing_init['TotalBaths'].describe()
housing_init.drop(columns =['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'], inplace= True)

#### Add 'OutdoorSF' as outdoor square feet, which represents the sum of wood deck, open porch, enclosed porch, 3 screen porch and screen porch square feet. 

In [60]:
# Add new column for outdoor square footage
housing_init['OutdoorSF'] = housing_init['WoodDeckSF'] + housing_init['OpenPorchSF'] + housing_init['EnclosedPorch'] + housing_init['3SsnPorch'] + housing_init['ScreenPorch'] 
housing_init['OutdoorSF'].describe()
housing_init.drop(columns =['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], inplace= True)

#### Convert month sold into calendar year quarter (Q1 for Jan-Mar, etc.)

In [61]:
# https://www.nar.realtor/blogs/economists-outlook/seasonality-in-the-housing-market
housing_init['MoSold'].describe()

# Cast quarter to new column from month 
housing_init['Quarter'] = np.nan
q = []
for month in housing_init['MoSold']:
    if month < 4:
        q.append('Q1')
    elif month < 7:
        q.append('Q2')
    elif month < 10:
        q.append('Q3')
    else:
        q.append('Q4')
housing_init['Quarter'] = q

housing_init.drop(columns =['MoSold'], inplace=True)
housing_init['Quarter'].groupby(housing_init['Quarter']).count()

Quarter
Q1    351
Q2    817
Q3    579
Q4    304
Name: Quarter, dtype: int64

#### Convert year to binary variable 'GFC', before and after 2008 (Great Financial Crisis) which negatively impacted real estate prices. 

In [62]:
housing_init['SalePrice'].groupby(housing_init['YrSold']).mean()

# Cast yr sold as before/after GFC
housing_init['GFC'] = np.nan
year = []
for yr in housing_init['YrSold']:
    if yr > 2009:
        year.append('0')
    else:
        year.append('1')
housing_init['GFC'] = year

housing_init.drop(columns =['YrSold'], inplace=True)
housing_init['GFC'].groupby(housing_init['GFC']).count()

GFC
0     234
1    1817
Name: GFC, dtype: int64

### Variable Types

In [63]:
# housing_init.info()
# Break out dummy vars. Convert variables to categorical (do this after TTS) 
housing_init_dummy = housing_init.drop(columns = ['TotalBsmtSF', 'GrLivArea', 'MiscVal', 'PoolArea', 'SalePrice', 'Age', 'OutdoorSF', 'TotRmsAbvGrd', 'GarageCars', 'Fireplaces', 'TotalBaths'])
# Break out numeric variables. Ensure in right variable type (float/int)
housing_init_num = housing_init[['TotalBsmtSF', 'GrLivArea', 'MiscVal', 'PoolArea', 'Age', 'OutdoorSF', 'TotRmsAbvGrd', 'GarageCars', 'Fireplaces', 'TotalBaths']]
# housing_init_num

In [64]:
# Create combined new data set for X variables
# https://pandas.pydata.org/docs/reference/api/pandas.concat.html
housing_xvars = pd.concat([housing_init_dummy, housing_init_num], axis=1)
# housing_xvars.head()
# housing_xvars.info()

# Preproccesing 

In [65]:
# Convert variables to categorical with OHE
oh = OneHotEncoder(sparse = False, drop = 'first')
housing_init_dummy_oh = oh.fit_transform(housing_init_dummy)
housing_xvars_oh = pd.concat([pd.DataFrame(housing_init_dummy_oh, columns = oh.get_feature_names_out()), housing_init_num], axis=1)

C:\Users\13478\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [66]:
# Define X and y variables
X = housing_xvars_oh
y = housing_init['SalePrice']

# TTS
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 95)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1538, 92) (1538,)
(513, 92) (513,)


In [67]:
# Add polynomial features
poly = PolynomialFeatures(include_bias = False, degree = 2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
X_poly = poly.fit_transform(X)

# Modeling

In [68]:
lr = LinearRegression()
lr.fit(X_train_poly, y_train)
print(lr.score(X_train_poly, y_train))
print(lr.score(X_test_poly, y_test))
preds = lr.predict(X_poly)
# preds.shape

0.992461657918613
-75.9714419528432


### Convert test data in same way as training data

In [69]:
housing_test0 = pd.read_csv('datasets/test-Copy1.csv')
housing_test0.columns = [n.replace(" ", "") for n in housing_test0.columns]

init_vars1 = get_var_name('datasets/initial_vars-Copy1.txt')
housing_test1 = housing_test0[init_vars1]

housing_test1 = housing_test1.fillna(0)

housing_test1['Age'] = 2011 - housing_test1['YearRemod/Add']
housing_test1.drop(columns =['YearRemod/Add'], inplace= True)

housing_test1['TotalBaths'] = housing_test1['BsmtFullBath'] + 0.5*housing_test1['BsmtHalfBath'] + housing_test1['FullBath'] + 0.5*housing_test1['HalfBath']
housing_test1.drop(columns =['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'], inplace= True)

housing_test1['OutdoorSF'] = housing_test1['WoodDeckSF'] + housing_test1['OpenPorchSF'] + housing_test1['EnclosedPorch'] + housing_test1['3SsnPorch'] + housing_test1['ScreenPorch'] 
housing_test1.drop(columns =['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], inplace= True)

housing_test1['Quarter'] = np.nan
q = []
for month in housing_test1['MoSold']:
    if month < 4:
        q.append('Q1')
    elif month < 7:
        q.append('Q2')
    elif month < 10:
        q.append('Q3')
    else:
        q.append('Q4')
housing_test1['Quarter'] = q
housing_test1.drop(columns =['MoSold'], inplace=True)

housing_test1['GFC'] = np.nan
year = []
for yr in housing_test1['YrSold']:
    if yr > 2009:
        year.append('0')
    else:
        year.append('1')
housing_test1['GFC'] = year
housing_test1.drop(columns =['YrSold'], inplace=True)

housing_test1_dummy = housing_test1.drop(columns = ['TotalBsmtSF', 'GrLivArea', 'MiscVal', 'PoolArea', 'Age', 'OutdoorSF', 'TotRmsAbvGrd', 'GarageCars', 'Fireplaces', 'TotalBaths'])
housing_test1_num = housing_test1[['TotalBsmtSF', 'GrLivArea', 'MiscVal', 'PoolArea', 'Age', 'OutdoorSF', 'TotRmsAbvGrd', 'GarageCars', 'Fireplaces', 'TotalBaths']]
oh = OneHotEncoder(sparse = False, drop = 'first')
housing_test1_dummy_oh = oh.fit_transform(housing_test1_dummy)
housing_xvars_test1_oh = pd.concat([pd.DataFrame(housing_test1_dummy_oh, columns = oh.get_feature_names_out()), housing_test1_num], axis=1)

C:\Users\13478\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Run model on testing data

In [70]:
# Need same columns present in test data and train data. 
# Identify missing columns bw test and training data then add as 0s where appropriate
# housing_xvars_test1_oh
miss_cols = housing_xvars_oh.columns.difference(housing_xvars_test1_oh.columns)

# Add missing columns to test data, fill with 0s
housing_xvars_test1_oh[miss_cols] = 0
# housing_xvars_oh.columns

# Sort columns in test data so in same order as training data
housing_xvars_test1_oh_sorted = pd.DataFrame(housing_xvars_test1_oh, columns = housing_xvars_oh.columns)

In [71]:
# Tranform test x vars to be poly
poly = PolynomialFeatures(include_bias = False, degree = 2)
housing_xvars_test1_oh_sorted_poly = poly.fit_transform(housing_xvars_test1_oh_sorted)

# Get predictions on test data
test_preds = lr.predict(housing_xvars_test1_oh_sorted_poly)

# Fit test data with model
housing_test0['SalePrice'] = test_preds
housing_test0

,Id,PID,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SalePrice
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2006,WD,26117.014262
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,8,2006,WD,208727.818351
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,9,2006,New,176338.323971
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2007,WD,591434.767862
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,185,0,NaN,NaN,NaN,0,7,2009,WD,157247.912768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2007,WD,550510.575723
874,1234,535126140,60,RL,90.0,14670,Pave,NaN,Reg,Lvl,...,0,0,NaN,MnPrv,NaN,0,8,2008,WD,205206.121027
875,1373,904100040,20,RL,55.0,8250,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,8,2008,WD,388537.758705
876,1672,527425140,20,RL,60.0,9000,Pave,NaN,Reg,Lvl,...,0,0,NaN,GdWo,NaN,0,5,2007,WD,163249.634955


In [72]:
# Save excel for submission
submission1 = housing_test0[['Id', 'SalePrice']]
submission1.set_index('Id', inplace = True)
submission1.to_csv('submission2.csv')
# submission2